In [1]:
import pandas as pd
import torch
import os
import numpy as np

In [2]:
def load_data_from_csv(file_path):
    df = pd.read_csv(file_path)
    labels = df[df.columns[-1]]
    features = df.drop(df.columns[-1], axis=1).values
    labels = np.where(labels, 1, 0)

    X = torch.tensor(features, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.int64)

    return X, y

In [3]:
from google.colab import drive, files
import os

drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/Colab Notebooks/mobile-proj/data"

Mounted at /content/drive


In [4]:
filepaths = []
for filename in os.listdir(data_path):
    filepath = os.path.join(data_path, filename)
    if os.path.isfile(filepath) and filepath[-3:] =="csv":
        filepaths.append(filepath)



In [5]:
file = filepaths[0]
X, y = load_data_from_csv(file)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train

tensor([[ 5.6751e-01,  3.0193e-01, -3.7781e-01,  ...,  3.8053e-01,
          3.6102e+00,  8.9541e-02],
        [ 7.1979e-01,  2.3663e-01, -1.3085e+00,  ...,  3.5244e-01,
          4.3336e+00, -5.6740e-02],
        [ 5.9576e-01,  2.4166e-01, -9.7428e-01,  ...,  3.8032e-01,
          4.3309e+00, -5.4250e-01],
        ...,
        [ 6.0913e-01,  2.5181e-01, -5.7048e-01,  ...,  3.4958e-01,
          3.8149e+00, -1.8996e-01],
        [ 5.7457e-01,  3.8256e-01, -1.2241e-01,  ...,  4.2521e-01,
          3.4903e+00, -8.7889e-04],
        [ 5.2467e-01,  2.6530e-01, -4.0556e-01,  ...,  4.6692e-01,
          3.7923e+00,  8.3161e-02]])

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

class PostureDetectionModel(tf.keras.Model):
    def __init__(self):
        super(PostureDetectionModel, self).__init__()

        self.fc1 = layers.Dense(64, activation='relu', input_shape=(99,))
        self.dropout1 = layers.Dropout(0.5)
        self.fc2 = layers.Dense(32, activation='relu')
        self.dropout2 = layers.Dropout(0.5)
        self.fc3 = layers.Dense(32, activation='relu')
        self.dropout3 = layers.Dropout(0.5)
        self.fc4 = layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

model = PostureDetectionModel()

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
from torch.utils.data import TensorDataset, DataLoader

X_train_np = np.array(X_train)
y_train_np = np.array(y_train)

model.fit(X_train_np, y_train_np, epochs=10, batch_size=5, shuffle=True, validation_split=0.2)

Epoch 1/10
35/35 [==============================] - 2s 10ms/step - loss: 0.7927 - accuracy: 0.5029 - val_loss: 0.6106 - val_accuracy: 0.7045
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 0.6734 - accuracy: 0.6127 - val_loss: 0.5625 - val_accuracy: 0.7045
Epoch 3/10
35/35 [==============================] - 0s 3ms/step - loss: 0.6612 - accuracy: 0.6590 - val_loss: 0.5877 - val_accuracy: 0.7045
Epoch 4/10
35/35 [==============================] - 0s 3ms/step - loss: 0.6766 - accuracy: 0.5780 - val_loss: 0.5560 - val_accuracy: 0.7045
Epoch 5/10
35/35 [==============================] - 0s 3ms/step - loss: 0.6499 - accuracy: 0.6647 - val_loss: 0.5094 - val_accuracy: 0.7045
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 0.5345 - accuracy: 0.7052 - val_loss: 0.4601 - val_accuracy: 0.7045
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 0.5711 - accuracy: 0.6705 - val_loss: 0.4383 - val_accuracy: 0.7727
Epoch 8/10
35/35 [=

In [25]:
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

test_loss, test_accuracy = model.evaluate(X_test_np, y_test_np, batch_size=5)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

11/11 [==============================] - 0s 5ms/step - loss: 0.4523 - accuracy: 0.7273
Test Accuracy: 72.73%


In [11]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [12]:
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y_test_np))

In [13]:
def representative_data_gen():
    for input_value, _ in test_dataset.batch(1).take(100):
        yield [input_value]

converter.representative_dataset = representative_data_gen

lite_quant_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [14]:
model_save_path = "/content/drive/MyDrive/Colab Notebooks/mobile-proj"
file_path = os.path.join(model_save_path, 'quantized_model.tflite')

with open(file_path, 'wb') as f:
    f.write(lite_quant_model)


In [24]:
interpreter = tf.lite.Interpreter(model_path=file_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

correct_predictions = 0
total_predictions = 0

for x, y_true in test_dataset:
    x = tf.cast(x, dtype=tf.float32)
    x = tf.expand_dims(x, axis=0)
    interpreter.set_tensor(input_details[0]['index'], x)

    interpreter.invoke()

    y_pred = interpreter.get_tensor(output_details[0]['index'])

    if round(y_pred[0][0]) == y_true.numpy():
        correct_predictions += 1
    total_predictions += 1

accuracy = correct_predictions / total_predictions
print(f'Quantized Model Accuracy: {accuracy * 100:.2f}%')

Quantized Model Accuracy: 70.91%


In [16]:
y_train

tensor([0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0])